In [ ]:
import numpy as np
import pandas as pd

In [ ]:
characteristics = pd.read_csv('/data/volume02/yuba/HC_CSV2.0/patient_birthday_gender_race.csv')

In [ ]:
demo_flagged_df = pd.read_csv('demo_flagged_df.csv')
lynch_flagged_df = pd.read_csv('lynch_flagged_df.csv')
multi_flagged_df = pd.read_csv('multi_flagged_df.csv')

In [ ]:
combined_df = pd.concat([demo_flagged_df, lynch_flagged_df, multi_flagged_df], ignore_index=True)

In [ ]:
combined_df['PATIENT_IDENTIFIER'].nunique()

In [ ]:
combined_df = combined_df.drop_duplicates(subset='PATIENT_IDENTIFIER')

In [ ]:
filtered_df = characteristics[characteristics['PATIENT_IDENTIFIER'].isin(combined_df['PATIENT_IDENTIFIER'])]

In [ ]:
# Mean Age and SD
ref_date = pd.Timestamp('2024-01-01')

filtered_df['DATE_BIRTH'] = pd.to_datetime(filtered_df['DATE_BIRTH'], errors='coerce')

#Calculate age as of 1st jan 2024
filtered_df['AGE_2024'] = (ref_date.year - filtered_df['DATE_BIRTH'].dt.year 
                           - ((filtered_df['DATE_BIRTH'].dt.month > ref_date.month) |
                             ((filtered_df['DATE_BIRTH'].dt.month == ref_date.month) &
                             (filtered_df['DATE_BIRTH'].dt.day > ref_date.day))
                             ).astype(int)
                          )

mean_age = filtered_df['AGE_2024'].mean()
sd_age = filtered_df['AGE_2024'].std()

print(f"Mean age: {mean_age:.2f}")
print(f"SD: {sd_age:.2f}")

In [ ]:
# Gender distribution 
unique_pat = filtered_df.drop_duplicates(subset='PATIENT_IDENTIFIER')

#unique_pat['GENDER'] = unique_pat['GENDER'].str.strip().str.capitalize()

total_pat = unique_pat.shape[0]
men_cnt = (unique_pat['GENDER'] == 'Male').sum()
men_percen = men_cnt * 100 / total_pat

print(f"Men Count: {men_cnt}")
print(f"Men Percentage: {men_percen}")

In [ ]:
# Race distribution
race_dist = unique_pat['RACE'].value_counts(dropna=False).to_frame(name='count')
race_dist['percent'] = (race_dist['count'] * 100 / total_pat).round(2)

race_dist